In [ ]:
!pip install -U pip
!pip install -U langchain-openai
!pip install -U langchain-community
!pip install openai
!pip install librosa
!pip install noisereduce
!pip install soundfile
!pip install pydub
!pip install langchain

from openai import OpenAI
import glob
import os
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
from google.colab import userdata
import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
from pydub import AudioSegment
import noisereduce as nr
import soundfile as sf
import json
from IPython.display import Markdown as md

In [ ]:
key = userdata.get('openai')

client = OpenAI(api_key=key)  # this is also the default, it can be omitted
path = 'dir'


In [ ]:
# sound_name
# sound_name = path.split('/')[-1]
# 拡張子削除
sound_type = path.split('.')[-1]
sound_name = path.split('.')[0]

# wavに変換する
!ffmpeg -i "{path}" "{sound_name}.wav"
path = f"{sound_name}.wav"
sound_type = f"wav"
# 音声ファイルを読み込む
audio, rate = librosa.load(path)

# ノイズ除去を実行
reduced_noise_audio = nr.reduce_noise(y=audio, sr=rate)
sf.write(f"{sound_name}_noisereduce.{sound_type}", reduced_noise_audio, rate)

path = f"{sound_name}_noisereduce.{sound_type}"

In [ ]:
def split_audio(file_path, segment_length_ms=60000):  # 60,000 ms is 1 minutes
    audio = AudioSegment.from_file(file_path)
    length_audio = len(audio)
    start = 0
    end = segment_length_ms
    parts = []

    while start < length_audio:
        # Ensure the end does not exceed the length of the audio
        end = min(start + segment_length_ms, length_audio)

        # Extract part of the audio
        part = audio[start:end]
        parts.append(part)

        # Move to the next segment
        start += segment_length_ms

    return parts

def whisper(path):
    audio_file= open(path, "rb")
    transcript = openai.Audio.transcribe("whisper-1", audio_file)
    transcript = client.audio.transcriptions.create()
    txt=transcript["text"]
    return txt

def analyze_html_with_gpt(text):
    content = f"""
    以下の文章から講義録を作製してください
    1. 講義のテーマ
    2. 見出し
    3. 内容
    の階層になるようにマークダウン形式でまとめてください
    # 講義:
    {text}
    # 注意事項
    ## 注意
    - ```markdown　のようにタグをつける例がありますが不適切です。
    - 必ず日本語で記載してください
    """
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": content},
        ])
    return response.choices[0].message.content

In [ ]:
# Replace 'your_audio_file.m4a' with the path to your m4a file
file_path = path
audio_parts = split_audio(file_path)
# Example of how to export the parts to separate files
for i, part in enumerate(audio_parts):
    part.export(f"part_{i}.wav", format="wav")
    # save dir
    print(f"part_{i}_noisereduced.wav")

In [ ]:
result = []
for wav in wav_files:
    print(f'Now processing {wav}')
    audio_file= open(wav, "rb")
    transcription = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file
    )
    print(transcription.text)
    result.append(transcription.text)

In [ ]:
summary = analyze_html_with_gpt(result)
md(summary)